In [2]:
#pip3 install twilio
#pip install bs4
#pip install tqdm

In [2]:
import os
from twilio.rest import Client
from twilio_config import TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN, PHONE_NUMBER, API_KEY_WAPI
import time

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

from datetime import datetime

In [22]:
print("Account SID", TWILIO_ACCOUNT_SID)

print("Token", TWILIO_AUTH_TOKEN)

Account SID ACe98cc22418d6aa9fab461892dffd38c3
Token 9ba08fbf2af24974260a98932d2090bb


# Armado de la URL

## Call

In [3]:
#Preparando Llamada o url 
query = 'Ciudad Juarez'
api_key = API_KEY_WAPI
url_clima = 'http://api.weatherapi.com/v1/forecast.json?key='+api_key+'&q='+query+'&days=2&aqi=no&alerts=no'

In [4]:
#Realizando llamada o peticion y conviertiendo resultado en formato JSON
response = requests.get(url_clima).json()

In [5]:
#Mostrando resultado
print(response)

{'location': {'name': 'Ciudad Juarez', 'region': 'Chihuahua', 'country': 'Mexico', 'lat': 31.73, 'lon': -106.48, 'tz_id': 'America/Ojinaga', 'localtime_epoch': 1685024732, 'localtime': '2023-05-25 9:25'}, 'current': {'last_updated_epoch': 1685024100, 'last_updated': '2023-05-25 09:15', 'temp_c': 22.0, 'temp_f': 71.6, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 4.3, 'wind_kph': 6.8, 'wind_degree': 210, 'wind_dir': 'SSW', 'pressure_mb': 1015.0, 'pressure_in': 29.96, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 17, 'cloud': 25, 'feelslike_c': 22.1, 'feelslike_f': 71.7, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 6.0, 'gust_mph': 2.5, 'gust_kph': 4.0}, 'forecast': {'forecastday': [{'date': '2023-05-25', 'date_epoch': 1684972800, 'day': {'maxtemp_c': 34.8, 'maxtemp_f': 94.6, 'mintemp_c': 21.8, 'mintemp_f': 71.2, 'avgtemp_c': 28.6, 'avgtemp_f': 83.4, 'maxwind_mph': 11.4, 'maxwind_kph': 18.4, 'totalpre

In [6]:
tf = open("data.txt", "w")
json.dump(response, tf)
tf.close()

In [7]:
response.keys()

dict_keys(['location', 'current', 'forecast'])

Armado del pronostico del clima Diario


In [8]:
response['forecast']['forecastday'][0]['day']

{'maxtemp_c': 23.2,
 'maxtemp_f': 73.8,
 'mintemp_c': 5.6,
 'mintemp_f': 42.1,
 'avgtemp_c': 14.3,
 'avgtemp_f': 57.7,
 'maxwind_mph': 5.1,
 'maxwind_kph': 8.3,
 'totalprecip_mm': 0.0,
 'totalprecip_in': 0.0,
 'totalsnow_cm': 0.0,
 'avgvis_km': 10.0,
 'avgvis_miles': 6.0,
 'avghumidity': 37.0,
 'daily_will_it_rain': 0,
 'daily_chance_of_rain': 0,
 'daily_will_it_snow': 0,
 'daily_chance_of_snow': 0,
 'condition': {'text': 'Partly cloudy',
  'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png',
  'code': 1003},
 'uv': 4.0}

In [9]:
#Temperatura Maxima y minima
date = response['forecast']['forecastday'][0]['date']
minTemp = response['forecast']['forecastday'][0]['day']['mintemp_c']
maxTemp = response['forecast']['forecastday'][0]['day']['maxtemp_c']
avgTemp = response['forecast']['forecastday'][0]['day']['avgtemp_c']
chanceOfRain = response['forecast']['forecastday'][0]['day']['daily_chance_of_rain'] 

In [10]:
messageWeather = "Hola Admin, este es el pronostico del clima para el dia de hoy: {}\n * Temperatura Minima: {} °C\n * Temperatura Maxima: {} °C\n * Temperatura Promedio: {} °C \n * Posibilidad de Lluvia: {}%".format(date, minTemp, maxTemp, avgTemp, chanceOfRain)

In [11]:
messageWeather

'Hola Admin, este es el pronostico del clima para el dia de hoy: 2023-02-25\n * Temperatura Minima: 5.6 °C\n * Temperatura Maxima: 23.2 °C\n * Temperatura Promedio: 14.3 °C \n * Posibilidad de Lluvia: 0%'

Creando Dataframe con la hora del posible pronostico de lluvia

In [12]:
def get_forecast(response, i):
    fecha = response['forecast']['forecastday'][0]['hour'][i]['time'].split()[0]
    hora = int(response['forecast']['forecastday'][0]['hour'][i]['time'].split()[1].split(":")[0])
    condicion = response['forecast']['forecastday'][0]['hour'][i]['condition']['text']
    temperatura = response['forecast']['forecastday'][0]['hour'][i]['temp_c']
    lluvia = response['forecast']['forecastday'][0]['hour'][i]['will_it_rain']
    prob_lluvia = response['forecast']['forecastday'][0]['hour'][i]['chance_of_rain']

    return fecha, hora, condicion, temperatura, lluvia, prob_lluvia


In [13]:
datos = []

#libreria tqdm permite visualizar una barra de carga
for i in tqdm(range(len(response['forecast']['forecastday'][0]['hour'])), colour = "green"):
    datos.append(get_forecast(response, i))

100%|██████████| 24/24 [00:00<?, ?it/s]


In [14]:
#Creando data Frame
col = ['fecha', 'hora', 'condicion', 'temperatura', 'lluvia', 'prob_lluvia']
df = pd.DataFrame(datos, columns=col)
df

,fecha,hora,condicion,temperatura,lluvia,prob_lluvia
0,2023-02-25,0,Partly cloudy,13.0,0,0
1,2023-02-25,1,Overcast,12.1,0,0
2,2023-02-25,2,Overcast,11.0,0,0
3,2023-02-25,3,Overcast,9.8,0,0
4,2023-02-25,4,Overcast,8.9,0,0
5,2023-02-25,5,Overcast,8.1,0,0
6,2023-02-25,6,Cloudy,7.3,0,0
7,2023-02-25,7,Partly cloudy,6.6,0,0
8,2023-02-25,8,Partly cloudy,5.8,0,0
9,2023-02-25,9,Partly cloudy,5.6,0,0


In [15]:
df_Final = df[df['prob_lluvia']>0]
df_Final

,fecha,hora,condicion,temperatura,lluvia,prob_lluvia


In [16]:
if df_Final.empty == False:
    messageWeather = messageWeather + "\n" + str(df_Final)

In [17]:
messageWeather

'Hola Admin, este es el pronostico del clima para el dia de hoy: 2023-02-25\n * Temperatura Minima: 5.6 °C\n * Temperatura Maxima: 23.2 °C\n * Temperatura Promedio: 14.3 °C \n * Posibilidad de Lluvia: 0%'

Enviando Mensaje por whats

In [19]:
# Download the helper library from https://www.twilio.com/docs/python/install
import os
from twilio.rest import Client

# Find your Account SID and Auth Token at twilio.com/console
# and set the environment variables. See http://twil.io/secure
account_sid = TWILIO_ACCOUNT_SID
auth_token = "92cd3b4491a17776c413d761807286db" #TWILIO_AUTH_TOKEN
client = Client(account_sid, auth_token)
fromWhatsAppNumber = 'whatsapp:+14155238886'
whatsappNumber = 'whatsapp:+5216565656768'

message = client.messages.create(
                    from_=fromWhatsAppNumber,
                    body=messageWeather,
                    to=whatsappNumber)

print('Mensaje Enviado' + message.sid)

Mensaje EnviadoSM8888b3107ca4c445eeacbbea548c681e


Mensaje de Texto con Twilio

In [20]:
# Download the helper library from https://www.twilio.com/docs/python/install
import os
from twilio.rest import Client


# Find your Account SID and Auth Token at twilio.com/console
# and set the environment variables. See http://twil.io/secure
account_sid = TWILIO_ACCOUNT_SID
auth_token = "92cd3b4491a17776c413d761807286db" #TWILIO_AUTH_TOKEN
client = Client(account_sid, auth_token)

message = client.messages.create(
         body= messageWeather,
         from_=PHONE_NUMBER,
         to='+526565656768'
         #to='+526562665660'
     )

print('Mensaje Enviado' + message.sid)

Mensaje EnviadoSMf63cade7a2551a63aa50d547c87e954d
